In [2]:
import anndata
import umap
import xgboost
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import pickle
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import display, Image, HTML
import mlflow
import re
from sklearn.model_selection import cross_val_score
regex = re.compile(r"perturbation_experiment_([^_]+)_level")

# base definition and original adata:
if not "adata" in locals():
    levels = ["0", "0.5", "1.0"]
    data_dir = "/data/gene_perturbation_colon_epithelial_scGPT_with_binning_1"
    file_pattern = data_dir + "/perturbation_experiment_{gene}_level_{level}"
    base_fname = file_pattern.format(gene = "JUND", level = "1.0")
    base_embeddings = np.load(f"{base_fname}.embeddings.npy")
    adata = anndata.read_h5ad(f"{base_fname}.h5ad")
    base_df = adata.obs[["tissue", "cell_type", 'sex', 'development_stage',"disease"]].copy()

In [3]:
import xgboost
x = base_embeddings

# shuffle the x on the y axis:
shuffled_x = x.copy()
np.random.shuffle(shuffled_x)
y = pd.Categorical(base_df["disease"]).codes
print(np.all(shuffled_x == x))
print(x.shape, shuffled_x.shape)

print("training model")
cv_model = cross_val_score(xgboost.XGBClassifier(), x, y, cv=5, scoring="accuracy", n_jobs=5)
print("training shuffled model")
cv_model_shuffled = cross_val_score(xgboost.XGBClassifier(), shuffled_x, y, cv=5, scoring="accuracy", n_jobs=5)

print(f"accuracy model: {cv_model.mean()} +/- {cv_model.std()}")
print(f"accuracy model shuffled: {cv_model_shuffled.mean()} +/- {cv_model_shuffled.std()}")


False
(97788, 512) (97788, 512)
training model
training shuffled model
accuracy model: 0.7442941876335241 +/- 0.041455252915455834
accuracy model shuffled: 0.516944825795031 +/- 0.001544958301107402
